In [ ]:
'''
using IEX API sandbox
equal weight stock portfolio programme
determine how many shares of each inputted stock
doesn't weigh in market captilization so small companies will have larger weight than bigger companies
'''

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter

In [ ]:
stocks = pd.read_csv('stocks.csv')#list of snp 500 stocks
stocks

In [ ]:
from apitoken import IEX_CLOUD_API_TOKEN

In [ ]:
#test loop
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

for stock in stocks['Ticker'][:5]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(pd.Series([stock, data['latestPrice'], data['marketCap'], 'N/A'], index = my_columns),
                           ignore_index = True)
final_dataframe   

In [ ]:
#use batch API calls for optimization, split list into sublists
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

#some stocks are not called in the batch_api_call_url for example 'ABC'
#therefore I have only included the first 4 stocks

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
            ], index = my_columns), ignore_index = True)


final_dataframe

In [ ]:
#calculating the number of shares to buy
while True:
    try:
        portfolio_size = float(input('Enter the value of your portfolio in £: '))
        if portfolio_size > 0:
            break
        print('Please Enter a valid number: ')
    except ValueError:
        print('Please Enter a valid number: ')

In [ ]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = round(position_size/final_dataframe.loc[i, 'Stock Price'], 6)
    
final_dataframe

In [ ]:
#formatting into an excel output
writer = pd.ExcelWriter('trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'trades', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

pound_format = writer.book.add_format({
        'num_format': '£0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

integer_format = writer.book.add_format({
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', pound_format],
    'C': ['Market Capitalization', pound_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['trades'].set_column(f'{column}:{column}', 18, string_format)

writer.save()